In [1]:
import pandas as pd 
import numpy as np 
from keras.utils.np_utils import to_categorical   

import recordlinker

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore', 'info')

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
iowa_matches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_matches.csv')
iowa_nonmatches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_nonmatches.csv')

In [4]:
iowa_matches.head()

,uid1915,fname1915,lname1915,fullname1915,yob1915,hhid,fname1940,lname1940,fullname1940,yob1940,uid-hhid
0,uid0910071227,donald d,cutler,donald d cutler,1911,19067,donald dean,cutler,donald dean cutler,1911,uid0910071227-19067
1,uid0063131339,homer,taylor,homer taylor,1912,71505,homer ellis,taylor,homer ellis taylor,1912,uid0063131339-71505
2,uid0044088276,earl,stearnes,earl stearnes,1899,109708,earl,stearns,earl stearns,1900,uid0044088276-109708
3,uid0067053130,theodore,hornaday,theodore hornaday,1904,108304,theodore i,harnaday,theodore i harnaday,1904,uid0067053130-108304
4,uid0066046148,jack r,turner,jack r turner,1907,105092,jack r,turner,jack r turner,1907,uid0066046148-105092


### String Embedding Example

In [42]:
from recordlinker.preprocess import embed_letters, embed_shingles, disembed_letters, disembed_shingles

name = 'kailin lu'
max_length = 12 

print('Embed Letters: \n', 
      embed_letters(name, max_length), 
      disembed_letters(embed_letters(name, max_length))) 

print('Embed Letters Normalized: \n', 
      embed_letters(name, max_length, normalize=True), 
      disembed_letters(embed_letters(name, max_length, normalize=True)))

print('Embed 2-Shingles: \n', 
      embed_shingles(name, max_length), 
      disembed_shingles(embed_shingles(name, max_length))) 

print('Embed 2-Shingles Normalized: \n', 
      embed_shingles(name, max_length, normalize=True), 
      disembed_shingles(embed_shingles(name, max_length, normalize=True)))

Embed Letters: 
 [11  1  9 12  9 14 27 12 21  0  0  0] kailin lu
Embed Letters Normalized: 
 [0.40740741 0.03703704 0.33333333 0.44444444 0.33333333 0.51851852
 1.         0.44444444 0.77777778 0.         0.         0.        ] kailin lu
Embed 2-Shingles: 
 [261   8 219 295 221 364 688 306   0   0   0   0] kailin lu
Embed 2-Shingles Normalized: 
 [0.35753425 0.0109589  0.3        0.40410959 0.30273973 0.49863014
 0.94246575 0.41917808 0.         0.         0.         0.        ] kailin lu


## Train and save autoencoders

### Dense

In [19]:
ORIG_LENGTH = 12
BATCH_SIZE = 32
ENCODE_DIM = [256, 128] 
DECODE_DIM = [128, 256]
LR = 1e-4
EPOCHS=301
LATENT_DIM = [2,4,8]

# Embed letters 
namesA = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1915',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters', 
                                                     normalize=True)
namesB = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1940',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters', 
                                                     normalize=True)

for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/dense_letter_{}/'.format(latent_dim)
    run_id = 'dense_{}'.format(latent_dim)
    vae = recordlinker.model.VAE(batch_size=BATCH_SIZE,
                                 orig_dim=ORIG_LENGTH, 
                                 latent_dim=latent_dim,
                                 encode_dim=ENCODE_DIM,
                                 decode_dim=DECODE_DIM,
                                 lr=LR)

    model, encoder, decoder = vae.train(namesA, namesB, 
                                        epochs=EPOCHS, 
                                        run_id=run_id,
                                        save_path=save_path,
                                        optimizer='adam', 
                                        tensorboard=True, 
                                        earlystop=True,
                                        earlystop_patience=6,
                                        reconstruct=True, 
                                        reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
enc_0 (Dense)                   (None, 256)          3328        input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (Dense)                   (None, 128)          32896       enc_0[0][0]                      
__________________________________________________________________________________________________
mu (Dense)                      (None, 2)            258         enc_1[0][0]                      
__________________________________________________________________________________________________
log_sigma 

3456/3456 [==============================] - 0s 78us/step - loss: 1.0556 - acc: 0.3961 - val_loss: 1.0836 - val_acc: 0.4039
Epoch 37/301
3456/3456 [==============================] - 0s 78us/step - loss: 1.0545 - acc: 0.3903 - val_loss: 1.0872 - val_acc: 0.4016
Epoch 38/301
3456/3456 [==============================] - 0s 78us/step - loss: 1.0531 - acc: 0.3935 - val_loss: 1.0854 - val_acc: 0.3993
Epoch 39/301
3456/3456 [==============================] - 0s 79us/step - loss: 1.0520 - acc: 0.3912 - val_loss: 1.0819 - val_acc: 0.3993
Epoch 40/301
3456/3456 [==============================] - 0s 78us/step - loss: 1.0474 - acc: 0.3973 - val_loss: 1.0773 - val_acc: 0.4062
Epoch 41/301
3456/3456 [==============================] - 0s 78us/step - loss: 1.0509 - acc: 0.3929 - val_loss: 1.0766 - val_acc: 0.4039
Sample Reconstructions:
{'Orig': 'ginger', 'Pred:': 'kgpljnb     '}
{'Orig': 'frein', 'Pred:': 'lqjkha      '}
{'Orig': 'soukip', 'Pred:': 'llnmknca    '}
{'Orig': 'laing', 'Pred:': 'ienjga  

3456/3456 [==============================] - 0s 105us/step - loss: 0.9723 - acc: 0.4580 - val_loss: 0.9952 - val_acc: 0.4537
Epoch 23/301
3456/3456 [==============================] - 0s 94us/step - loss: 0.9646 - acc: 0.4511 - val_loss: 0.9975 - val_acc: 0.4433
Epoch 24/301
3456/3456 [==============================] - 0s 93us/step - loss: 0.9664 - acc: 0.4497 - val_loss: 1.0135 - val_acc: 0.4387
Epoch 25/301
3456/3456 [==============================] - 0s 85us/step - loss: 0.9638 - acc: 0.4517 - val_loss: 1.0007 - val_acc: 0.4502
Epoch 26/301
3456/3456 [==============================] - 0s 87us/step - loss: 0.9643 - acc: 0.4476 - val_loss: 0.9875 - val_acc: 0.4549
Epoch 27/301
3456/3456 [==============================] - 0s 82us/step - loss: 0.9591 - acc: 0.4549 - val_loss: 0.9935 - val_acc: 0.4479
Epoch 28/301
3456/3456 [==============================] - 0s 83us/step - loss: 0.9511 - acc: 0.4566 - val_loss: 0.9870 - val_acc: 0.4745
Epoch 29/301
3456/3456 [=============================

Sample Reconstructions:
{'Orig': 'haupert', 'Pred:': 'gbpnfpmeba  '}
{'Orig': 'blades', 'Pred:': 'kidgdod     '}
{'Orig': 'taylor', 'Pred:': 'kevkooba    '}
{'Orig': 'latham', 'Pred:': 'hcqidka     '}
{'Orig': 'mueller', 'Pred:': 'kqcklelcba  '}
Epoch 22/301
3456/3456 [==============================] - 0s 82us/step - loss: 0.7857 - acc: 0.5781 - val_loss: 0.8578 - val_acc: 0.5787
Epoch 23/301
3456/3456 [==============================] - 0s 80us/step - loss: 0.7851 - acc: 0.5883 - val_loss: 0.8381 - val_acc: 0.6053
Epoch 24/301
3456/3456 [==============================] - 0s 79us/step - loss: 0.7804 - acc: 0.5749 - val_loss: 0.8369 - val_acc: 0.6157
Epoch 25/301
3456/3456 [==============================] - 0s 84us/step - loss: 0.7733 - acc: 0.5836 - val_loss: 0.8383 - val_acc: 0.5718
Epoch 26/301
3456/3456 [==============================] - 0s 83us/step - loss: 0.7703 - acc: 0.5833 - val_loss: 0.8277 - val_acc: 0.5891
Epoch 27/301
3456/3456 [==============================] - 0s 80us/ste

In [ ]:
# Embed shingles
ORIG_LENGTH = 12
BATCH_SIZE = 32
ENCODE_DIM = [256, 128] 
DECODE_DIM = [128, 256]
LR = 1e-4
EPOCHS=301
EMBED_TYPE = 'shingles'
LATENT_DIM = [2,4,8]

# Embed letters 
namesA = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1915',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type=EMBED_TYPE, 
                                                     normalize=True)
namesB = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1940',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type=EMBED_TYPE, 
                                                     normalize=True)

for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/dense_shingle_{}_iowa_last/'.format(latent_dim)
    run_id = 'dense_{}'.format(latent_dim)
    vae = recordlinker.model.VAE(batch_size=BATCH_SIZE,
                                 orig_dim=ORIG_LENGTH, 
                                 latent_dim=latent_dim,
                                 encode_dim=ENCODE_DIM,
                                 decode_dim=DECODE_DIM,
                                 lr=LR)

    model, encoder, decoder = vae.train(namesA, namesB, 
                                        epochs=EPOCHS, 
                                        run_id=run_id,
                                        save_path=save_path,
                                        optimizer='adam', 
                                        tensorboard=True, 
                                        earlystop=True,
                                        earlystop_patience=6,
                                        reconstruct=True, 
                                        reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
enc_0 (Dense)                   (None, 256)          3328        input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (Dense)                   (None, 128)          32896       enc_0[0][0]                      
__________________________________________________________________________________________________
mu (Dense)                      (None, 2)            258         enc_1[0][0]                      
__________________________________________________________________________________________________
log_sigma 

3456/3456 [==============================] - 0s 80us/step - loss: 0.9020 - acc: 0.5428 - val_loss: 0.9464 - val_acc: 0.5440
Epoch 37/301
3456/3456 [==============================] - 0s 80us/step - loss: 0.8947 - acc: 0.5489 - val_loss: 0.9313 - val_acc: 0.5243
Epoch 38/301
3456/3456 [==============================] - 0s 80us/step - loss: 0.8951 - acc: 0.5448 - val_loss: 0.9270 - val_acc: 0.5266
Epoch 39/301
3456/3456 [==============================] - 0s 89us/step - loss: 0.9010 - acc: 0.5347 - val_loss: 0.9290 - val_acc: 0.5394
Epoch 40/301
3456/3456 [==============================] - 0s 83us/step - loss: 0.8962 - acc: 0.5420 - val_loss: 0.9198 - val_acc: 0.5324
Epoch 41/301
3456/3456 [==============================] - 0s 81us/step - loss: 0.8909 - acc: 0.5411 - val_loss: 0.9410 - val_acc: 0.5197
Sample Reconstructions:
{'Orig': 'na ', 'Pred:': 'fcxdb       '}
{'Orig': 'ktmd', 'Pred:': 'jnjda       '}
{'Orig': 'enrrd', 'Pred:': 'jhplgb      '}
{'Orig': 'hmfl ', 'Pred:': 'kmjica      '

Train on 3456 samples, validate on 864 samples
Epoch 1/301
3456/3456 [==============================] - 0s 129us/step - loss: 3.6774 - acc: 0.3455 - val_loss: 1.8294 - val_acc: 0.3727
Sample Reconstructions:
{'Orig': 'ltqoh', 'Pred:': 'jjmifdbaaaaa'}
{'Orig': 'b nod', 'Pred:': 'ijmifdbaaa  '}
{'Orig': 'gt rl ', 'Pred:': 'jjmifdbaaaaa'}
{'Orig': 'khqbnl', 'Pred:': 'jjmjgecbaaaa'}
{'Orig': 'qhbj', 'Pred:': 'ijmifcaaa   '}
Epoch 2/301
3456/3456 [==============================] - 0s 84us/step - loss: 1.4844 - acc: 0.3993 - val_loss: 1.2601 - val_acc: 0.5370
Epoch 3/301
3456/3456 [==============================] - 0s 76us/step - loss: 1.0885 - acc: 0.6432 - val_loss: 1.0631 - val_acc: 0.6400
Epoch 4/301
3456/3456 [==============================] - 0s 80us/step - loss: 0.9818 - acc: 0.6412 - val_loss: 0.9977 - val_acc: 0.6076
Epoch 5/301
3456/3456 [==============================] - 0s 76us/step - loss: 0.9302 - acc: 0.6285 - val_loss: 0.9670 - val_acc: 0.5972
Epoch 6/301
3456/3456 [=========

### Convolutional

In [327]:
vae_conv = recordlinker.model.ConvolutionalVAE(batch_size=BATCH_SIZE, 
                                               orig_dim=ORIG_LENGTH,
                                               latent_dim=LATENT_DIM, 
                                               lr=LR)

model_conv, encoder_conv, decoder_conv = vae_conv.train(namesA, namesB,
                                                       epochs=EPOCHS, 
                                                       run_id='conv',
                                                       save_path='/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/conv_model',
                                                       optimizer='rmsprop', 
                                                       earlystop=False, 
                                                       earlystop_patience=10, 
                                                       reconstruct=True, 
                                                       reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (32, 12)             0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (32, 12, 1)          0           input_1[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv1D)                  (32, 12, 16)         64          reshape_1[0][0]                  
__________________________________________________________________________________________________
conv1 (Conv1D)                  (32, 12, 16)         784         conv0[0][0]                      
__________________________________________________________________________________________________
flatten (F

3456/3456 [==============================] - 0s 99us/step - loss: 4360.4917 - acc: 0.7025 - val_loss: 4505.7358 - val_acc: 0.6771
Epoch 75/500
3456/3456 [==============================] - 0s 96us/step - loss: 4360.0757 - acc: 0.7072 - val_loss: 4499.9478 - val_acc: 0.6690
Epoch 76/500
3456/3456 [==============================] - 0s 95us/step - loss: 4360.1210 - acc: 0.7031 - val_loss: 4502.7256 - val_acc: 0.6562
Epoch 77/500
3456/3456 [==============================] - 0s 95us/step - loss: 4358.7065 - acc: 0.6991 - val_loss: 4503.7463 - val_acc: 0.5995
Epoch 78/500
3456/3456 [==============================] - 0s 94us/step - loss: 4359.7082 - acc: 0.7075 - val_loss: 4507.3795 - val_acc: 0.6481
Epoch 79/500
3456/3456 [==============================] - 0s 101us/step - loss: 4358.4876 - acc: 0.7011 - val_loss: 4501.9432 - val_acc: 0.6956
Epoch 80/500
3456/3456 [==============================] - 0s 105us/step - loss: 4359.2541 - acc: 0.7034 - val_loss: 4498.6060 - val_acc: 0.6863
Epoch 81/5

Sample Reconstructions:
{'Orig': 'curtis', 'Pred:': 'dwrtkwa     '}
{'Orig': 'decker', 'Pred:': 'iihpizd     '}
{'Orig': 'huser', 'Pred:': 'hwqetc      '}
{'Orig': 'mammen', 'Pred:': 'tcsqhta     '}
{'Orig': 'mosier', 'Pred:': 'qqsjgvb     '}
Epoch 172/500
3456/3456 [==============================] - 0s 106us/step - loss: 4345.2168 - acc: 0.7216 - val_loss: 4499.9083 - val_acc: 0.6713
Epoch 173/500
3456/3456 [==============================] - 0s 111us/step - loss: 4345.5584 - acc: 0.7257 - val_loss: 4515.5591 - val_acc: 0.6806
Epoch 174/500
3456/3456 [==============================] - 0s 106us/step - loss: 4346.8502 - acc: 0.7208 - val_loss: 4517.4565 - val_acc: 0.6979
Epoch 175/500
3456/3456 [==============================] - 0s 96us/step - loss: 4345.5365 - acc: 0.7112 - val_loss: 4497.9045 - val_acc: 0.6898
Epoch 176/500
3456/3456 [==============================] - 0s 103us/step - loss: 4345.9336 - acc: 0.7130 - val_loss: 4501.7328 - val_acc: 0.6840
Epoch 177/500
3456/3456 [========

Epoch 220/500
3456/3456 [==============================] - 0s 106us/step - loss: 4344.1438 - acc: 0.7112 - val_loss: 4523.9492 - val_acc: 0.6667
Epoch 221/500
3456/3456 [==============================] - 0s 107us/step - loss: 4342.7182 - acc: 0.7179 - val_loss: 4502.1957 - val_acc: 0.6782
Sample Reconstructions:
{'Orig': 'feay', 'Pred:': 'jhgzi       '}
{'Orig': 'mammen', 'Pred:': 'sbvqjsa     '}
{'Orig': 'delong', 'Pred:': 'fhtttka     '}
{'Orig': 'buelow', 'Pred:': 'cyilrya     '}
{'Orig': 'dunkerson', 'Pred:': 'evqnhwyysa  '}
Epoch 222/500
3456/3456 [==============================] - 0s 103us/step - loss: 4342.7692 - acc: 0.7159 - val_loss: 4515.6717 - val_acc: 0.6898
Epoch 223/500
3456/3456 [==============================] - 0s 104us/step - loss: 4344.1931 - acc: 0.7144 - val_loss: 4503.1678 - val_acc: 0.6493
Epoch 224/500
3456/3456 [==============================] - 0s 102us/step - loss: 4341.2336 - acc: 0.7240 - val_loss: 4509.9002 - val_acc: 0.6725
Epoch 225/500
3456/3456 [=====

Epoch 268/500
3456/3456 [==============================] - 0s 102us/step - loss: 4341.8766 - acc: 0.7098 - val_loss: 4519.0472 - val_acc: 0.7025
Epoch 269/500
3456/3456 [==============================] - 0s 102us/step - loss: 4341.1016 - acc: 0.7150 - val_loss: 4506.8205 - val_acc: 0.6458
Epoch 270/500
3456/3456 [==============================] - 0s 104us/step - loss: 4342.5573 - acc: 0.7124 - val_loss: 4512.1778 - val_acc: 0.6921
Epoch 271/500
3456/3456 [==============================] - 0s 103us/step - loss: 4340.5042 - acc: 0.7135 - val_loss: 4498.3647 - val_acc: 0.6748
Sample Reconstructions:
{'Orig': 'bash', 'Pred:': 'cbyoa      '}
{'Orig': 'steiner', 'Pred:': 'uvggoes     '}
{'Orig': 'metcalf', 'Pred:': 'qhyfbtla    '}
{'Orig': 'coon', 'Pred:': 'dtvta       '}
{'Orig': 'lee', 'Pred:': 'rgia       '}
Epoch 272/500
3456/3456 [==============================] - 0s 108us/step - loss: 4341.5807 - acc: 0.7083 - val_loss: 4520.6840 - val_acc: 0.6863
Epoch 273/500
3456/3456 [=============

3456/3456 [==============================] - 0s 104us/step - loss: 4342.0326 - acc: 0.7121 - val_loss: 4522.8541 - val_acc: 0.6956
Epoch 365/500
3456/3456 [==============================] - 0s 101us/step - loss: 4340.1212 - acc: 0.7046 - val_loss: 4511.8874 - val_acc: 0.6667
Epoch 366/500
3456/3456 [==============================] - 0s 100us/step - loss: 4339.3681 - acc: 0.7078 - val_loss: 4507.7069 - val_acc: 0.6759
Epoch 367/500
3456/3456 [==============================] - 0s 94us/step - loss: 4339.2919 - acc: 0.7147 - val_loss: 4520.0563 - val_acc: 0.6933
Epoch 368/500
3456/3456 [==============================] - 0s 96us/step - loss: 4340.6001 - acc: 0.7028 - val_loss: 4507.4364 - val_acc: 0.6678
Epoch 369/500
3456/3456 [==============================] - 0s 100us/step - loss: 4341.4853 - acc: 0.7130 - val_loss: 4507.2915 - val_acc: 0.6100
Epoch 370/500
3456/3456 [==============================] - 0s 92us/step - loss: 4343.0433 - acc: 0.7089 - val_loss: 4528.4233 - val_acc: 0.6840
Ep

3456/3456 [==============================] - 0s 105us/step - loss: 4337.8419 - acc: 0.6962 - val_loss: 4532.7636 - val_acc: 0.6632
Epoch 463/500
3456/3456 [==============================] - 0s 99us/step - loss: 4337.6564 - acc: 0.6950 - val_loss: 4532.7661 - val_acc: 0.6887
Epoch 464/500
3456/3456 [==============================] - 0s 105us/step - loss: 4338.2586 - acc: 0.7069 - val_loss: 4517.3042 - val_acc: 0.6400
Epoch 465/500
3456/3456 [==============================] - 0s 100us/step - loss: 4336.3308 - acc: 0.7046 - val_loss: 4510.9014 - val_acc: 0.6678
Epoch 466/500
3456/3456 [==============================] - 0s 96us/step - loss: 4336.9874 - acc: 0.6976 - val_loss: 4516.6994 - val_acc: 0.6690
Epoch 467/500
3456/3456 [==============================] - 0s 99us/step - loss: 4336.8198 - acc: 0.6959 - val_loss: 4510.4693 - val_acc: 0.6435
Epoch 468/500
3456/3456 [==============================] - 0s 101us/step - loss: 4338.7046 - acc: 0.7072 - val_loss: 4539.6097 - val_acc: 0.6690
Ep

### LSTM

In [315]:
# Train 
# One hot encoding of names
ORIG_LENGTH = 12
classes = 28
namesA = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1915',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters', 
                                                     normalize=False)

namesB = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1940',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters',
                                                     normalize=False)

namesA_onehot = to_categorical(namesA, classes) 
namesB_onehot = to_categorical(namesB, classes)
print(namesA_onehot.shape, namesB_onehot.shape)

LR = 1e-3
BATCH_SIZE = 32
lstm_vae = recordlinker.model.LSTMVAE(batch_size=BATCH_SIZE, 
                                      timesteps=ORIG_LENGTH, 
                                      orig_dim=classes,
                                      latent_dim=8,
                                      encode_dim=64, 
                                      decode_dim=64,
                                      lr=LR) 
EPOCHS = 500
model_lstm, model_encoder, model_decoder = lstm_vae.train(namesA_onehot, namesB_onehot, 
                                                          epochs=EPOCHS, 
                                                          run_id='lstm_vae',
                                                          save_path='/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/test_lstm', 
                                                          earlystop=False,
                                                          tensorboard=True, 
                                                          reconstruct=True, 
                                                          reconstruct_display=10)

(4320, 12, 28) (4320, 12, 28)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12, 28)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 12, 64)       23808       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 12, 64)       33024       lstm_1[0][0]                     
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 768)          0           lstm_2[0][0]                     
_______________________________________________________________________________

3456/3456 [==============================] - 3s 977us/step - loss: 12.7501 - acc: 0.6824 - val_loss: 13.3846 - val_acc: 0.6713
Epoch 33/500
3456/3456 [==============================] - 3s 946us/step - loss: 12.7028 - acc: 0.6817 - val_loss: 13.3196 - val_acc: 0.6764
Epoch 34/500
3456/3456 [==============================] - 3s 961us/step - loss: 12.5320 - acc: 0.6853 - val_loss: 13.1931 - val_acc: 0.6763
Epoch 35/500
3456/3456 [==============================] - 3s 993us/step - loss: 12.4432 - acc: 0.6890 - val_loss: 13.0715 - val_acc: 0.6816
Epoch 36/500
3456/3456 [==============================] - 3s 963us/step - loss: 12.3004 - acc: 0.6929 - val_loss: 13.0580 - val_acc: 0.6798
Epoch 37/500
3456/3456 [==============================] - 3s 959us/step - loss: 12.2307 - acc: 0.6931 - val_loss: 12.8632 - val_acc: 0.6849
Epoch 38/500
3456/3456 [==============================] - 3s 981us/step - loss: 12.0975 - acc: 0.6975 - val_loss: 12.8856 - val_acc: 0.6865
Epoch 39/500
3456/3456 [=========

3456/3456 [==============================] - 3s 985us/step - loss: 7.1494 - acc: 0.8212 - val_loss: 11.5199 - val_acc: 0.7582
Epoch 135/500
3456/3456 [==============================] - 3s 994us/step - loss: 6.9353 - acc: 0.8291 - val_loss: 11.4534 - val_acc: 0.7623
Epoch 136/500
3456/3456 [==============================] - 3s 986us/step - loss: 6.8308 - acc: 0.8329 - val_loss: 11.3322 - val_acc: 0.7660
Epoch 137/500
3456/3456 [==============================] - 4s 1ms/step - loss: 6.7489 - acc: 0.8345 - val_loss: 11.5990 - val_acc: 0.7639
Epoch 138/500
3456/3456 [==============================] - 3s 1ms/step - loss: 6.8586 - acc: 0.8317 - val_loss: 11.3783 - val_acc: 0.7663
Epoch 139/500
3456/3456 [==============================] - 3s 966us/step - loss: 6.8030 - acc: 0.8317 - val_loss: 11.3868 - val_acc: 0.7674
Epoch 140/500
3456/3456 [==============================] - 3s 1ms/step - loss: 6.7341 - acc: 0.8350 - val_loss: 11.4937 - val_acc: 0.7641
Epoch 141/500
3456/3456 [===============

3456/3456 [==============================] - 4s 1ms/step - loss: 4.5718 - acc: 0.8916 - val_loss: 12.1359 - val_acc: 0.7943
Epoch 237/500
3456/3456 [==============================] - 3s 967us/step - loss: 4.5767 - acc: 0.8911 - val_loss: 12.3050 - val_acc: 0.7944
Epoch 238/500
3456/3456 [==============================] - 3s 989us/step - loss: 4.5221 - acc: 0.8925 - val_loss: 12.1637 - val_acc: 0.7926
Epoch 239/500
3456/3456 [==============================] - 4s 1ms/step - loss: 4.5700 - acc: 0.8921 - val_loss: 12.0419 - val_acc: 0.7942
Epoch 240/500
3456/3456 [==============================] - 4s 1ms/step - loss: 4.4814 - acc: 0.8946 - val_loss: 12.0884 - val_acc: 0.7973
Epoch 241/500
3456/3456 [==============================] - 4s 1ms/step - loss: 4.5634 - acc: 0.8913 - val_loss: 12.0828 - val_acc: 0.7966
Sample Reconstructions:
{'Orig': 'reinart', 'Pred:': 'seinart'}
{'Orig': 'fink', 'Pred:': 'kink'}
{'Orig': 'heskett', 'Pred:': 'setkett'}
{'Orig': 'denham', 'Pred:': 'denham'}
{'Orig

3456/3456 [==============================] - 3s 989us/step - loss: 3.0815 - acc: 0.9275 - val_loss: 12.9504 - val_acc: 0.8145
Epoch 339/500
3456/3456 [==============================] - 3s 980us/step - loss: 2.9800 - acc: 0.9302 - val_loss: 13.0167 - val_acc: 0.8118
Epoch 340/500
3456/3456 [==============================] - 3s 975us/step - loss: 3.0730 - acc: 0.9280 - val_loss: 12.9209 - val_acc: 0.8155
Epoch 341/500
3456/3456 [==============================] - 3s 985us/step - loss: 2.9853 - acc: 0.9302 - val_loss: 12.9437 - val_acc: 0.8166
Sample Reconstructions:
{'Orig': 'kuper', 'Pred:': 'puper'}
{'Orig': 'disher', 'Pred:': 'discher'}
{'Orig': 'miller', 'Pred:': 'miller'}
{'Orig': 'mccarty', 'Pred:': 'mccarty'}
{'Orig': 'deppel', 'Pred:': 'dippel'}
Epoch 342/500
3456/3456 [==============================] - 3s 959us/step - loss: 3.0415 - acc: 0.9287 - val_loss: 12.8765 - val_acc: 0.8156
Epoch 343/500
3456/3456 [==============================] - 3s 971us/step - loss: 3.4934 - acc: 0.91

3456/3456 [==============================] - 3s 971us/step - loss: 2.1869 - acc: 0.9505 - val_loss: 14.1033 - val_acc: 0.8255
Epoch 441/500
3456/3456 [==============================] - 3s 988us/step - loss: 2.0988 - acc: 0.9520 - val_loss: 13.7955 - val_acc: 0.8288
Sample Reconstructions:
{'Orig': 'cosner', 'Pred:': 'cosner'}
{'Orig': 'krouse', 'Pred:': 'krouse'}
{'Orig': 'griesse', 'Pred:': 'driesse'}
{'Orig': 'sass', 'Pred:': 'sass'}
{'Orig': 'james', 'Pred:': 'james'}
Epoch 442/500
3456/3456 [==============================] - 3s 957us/step - loss: 2.0670 - acc: 0.9525 - val_loss: 14.0077 - val_acc: 0.8281
Epoch 443/500
3456/3456 [==============================] - 3s 985us/step - loss: 2.1138 - acc: 0.9513 - val_loss: 14.0243 - val_acc: 0.8239
Epoch 444/500
3456/3456 [==============================] - 3s 976us/step - loss: 2.3280 - acc: 0.9451 - val_loss: 14.1226 - val_acc: 0.8214
Epoch 445/500
3456/3456 [==============================] - 3s 989us/step - loss: 2.3969 - acc: 0.9443 - 

3456/3456 [==============================] - 3s 968us/step - loss: 3.5502 - acc: 0.9153 - val_loss: 14.5776 - val_acc: 0.8094
Sample Reconstructions:
{'Orig': 'hull', 'Pred:': 'hull'}
{'Orig': 'hibbs', 'Pred:': 'hieis'}
{'Orig': 'koos', 'Pred:': 'koos'}
{'Orig': 'erbst', 'Pred:': 'egbst'}
{'Orig': 'dahl', 'Pred:': 'dahll'}
Epoch 492/500
3456/3456 [==============================] - 3s 952us/step - loss: 2.7155 - acc: 0.9365 - val_loss: 14.5552 - val_acc: 0.8223
Epoch 493/500
3456/3456 [==============================] - 3s 970us/step - loss: 2.2332 - acc: 0.9486 - val_loss: 14.2958 - val_acc: 0.8269
Epoch 494/500
3456/3456 [==============================] - 3s 984us/step - loss: 1.9296 - acc: 0.9558 - val_loss: 14.5301 - val_acc: 0.8289
Epoch 495/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.8781 - acc: 0.9575 - val_loss: 14.5933 - val_acc: 0.8301
Epoch 496/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.9197 - acc: 0.9562 - val_loss: 14.51